<a href="https://colab.research.google.com/github/tkuri/irradiance_estimation/blob/master/brightness_arbitraryL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git Clone

In [ ]:
%cd /content/drive/My\ Drive/Colabdata/
!git clone https://github.com/tkuri/irradiance_estimation.git

# Irradiance estimation only (Baseline)

In [ ]:
# Move to project directory
%cd /content/drive/My\ Drive/Colabdata/irradiance_estimation

# Import and install modules
import subprocess
from util_eval import eval_data, rem_checkmodel
!pip install -r requirements.txt
!git pull

# Experimental setup
traintest_mode = 0 # 0:train&test, 1:train, 2:test
scene = 'o2l1_mv_scl_ref_tm' # Scene
datamodel = [ # Dataset & Model
             ('cbox_tgt+lgt_to_irrgt', 'unet256_in2'),
             ('cbox_tgt+lgt_to_irrgt', 'pix2pix_in2'),
             ]
name_suffix = '_debug_0721.00' # Suffix of experimental name


def train_proc():
  !python train.py --dataroot ./datasets/{scene}/{data} --name ./{scene}/{name}{name_suffix} --model {model} --dataset_mode aligned3_tm_max_rnd --show_multi

def test_proc():
  !python test.py --dataroot ./datasets/{scene}/{data} --name ./{scene}/{name}{name_suffix} --model {model} --dataset_mode aligned3_tm_max_rnd --show_multi
  # Evaluate irradiance data
  try:
    eval_data('{}/{}'.format(scene, name))
  except OSError as e:
    pass

for data, model in datamodel:
  name = '{}_{}'.format(data, model)
  if traintest_mode==0:
    train_proc()
    test_proc()
  elif traintest_mode==1:
    train_proc()
  else:
    test_proc()

# Irradiance estimation only (LTM)

In [ ]:
# Move to project directory
%cd /content/drive/My\ Drive/Colabdata/irradiance_estimation

# Import and install modules
import subprocess
from util_eval import eval_data, rem_checkmodel
!pip install -r requirements.txt
!git pull

# Experimental setup
traintest_mode = 0 # 0:train&test, 1:train, 2:test
scene = 'o2l1_mv_scl_ref_tm' # Scene
datamodel = [ # Dataset & Model
             ('cbox_tgt+lgt_to_irrgt', 'pix2pix_tm_reg'),
             ]
name_suffix = '_debug_0721.00' # Suffix of experimental name
reglam = 10 # If set 0, no regularization
light_res = 5

def train_proc():
  !python train.py --dataroot ./datasets/{scene}/{data} --name ./{scene}/{name}{name_suffix} --model {model} --dataset_mode aligned3_tm_max_rnd --light_res {light_res} --show_multi --lambda_LTMReg {reglam}

def test_proc():
  !python test.py --dataroot ./datasets/{scene}/{data} --name ./{scene}/{name}{name_suffix} --model {model} --dataset_mode aligned3_tm_max_rnd --light_res {light_res} --show_multi
  # Evaluate irradiance data
  try:
    eval_data('{}/{}'.format(scene, name))
  except OSError as e:
    pass

for data, model in datamodel:
  name = '{}_{}'.format(data, model)
  if traintest_mode==0:
    train_proc()
    test_proc()
  elif traintest_mode==1:
    train_proc()
  else:
    test_proc()

# Finding brightest portions

In [ ]:
%cd /content/drive/My\ Drive/Colabdata/irradiance_estimation
!pip install -r requirements.txt
!git pull

traintest_mode = 0 # 0:train&test, 1:train, 2:test

gpu = 0
data = './datasets/o2l1_mv_scl_ref_tm/cbox_tgt+lgt_to_irrgt'
scene = 'o2l1_mv_scl_ref_tm'
model = 'brightest_mul_inL' # Baseline
# model = 'brightest_mul_tm_cas' # Calc LTM

suffix = '_debug_0721.00'
dataset = 'aligned3_bp_tm_max_rnd'
result_name = model+suffix+'_bp_eval'

command = '--dataroot {} --name ./{}/{}{} --model {} --dataset_mode {}'.format(data, scene, model, suffix, model, dataset)
command += ' --show_multi'
command += ' --light_res 5'
# command += ' --max_dataset_size 10'
# command += ' --cat_In'
# command += ' --disp_brighest_info'
# command += ' --shading_norm'
# command += ' --no_mask'

train_command = command + ' --save_latest_freq 100 --n_epochs 5 --n_epochs_decay 5'
train_command += ' --save_epoch_freq 2'
train_command += ' --display_freq 20 --update_html_freq 20 --print_freq 5'
# train_command += ' --continue_train --epoch_count 3'
test_command = command + ' --num_test 100 --test_mode 1'
test_command += ' --no_save_image'
test_command += ' --show_multi'
# test_command += ' --re_index'

if traintest_mode==0:
    !python train.py {train_command}
    !python test_bp.py {test_command}
elif traintest_mode==1:
    !python train.py {train_command}
else:
    !python test_bp.py {test_command}